In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and explore the dataset
df = pd.read_csv("lab_11_bridge_data.csv")
print(df.info())
print(df.head())
print(df.isnull().sum())  # Check for missing values

# One-hot encode the Material column
df = pd.get_dummies(df, columns=["Material"], drop_first=True)

# Define features and target variable
X = df.drop(columns=["Bridge_ID", "Max_Load_Tons"])
y = df["Max_Load_Tons"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Normalize the target variable
y_mean, y_std = y_train.mean(), y_train.std()
y_train_scaled = (y_train - y_mean) / y_std
y_test_scaled = (y_test - y_mean) / y_std

# Save the preprocessing pipeline
preprocess = {"scaler": scaler, "y_mean": y_mean, "y_std": y_std}
with open("preprocess.pkl", "wb") as f:
    pickle.dump(preprocess, f)

# Define the ANN model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=64, 
                    callbacks=[early_stop])

# Plot training history
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test_scaled, y_test_scaled)
print(f"Test MAE: {test_mae * y_std:.2f} tons")

# Save the trained model
model.save("tf_bridge_model.h5")

ModuleNotFoundError: No module named 'tensorflow'